In [1]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [2]:
def tutorial8():
    tile13 = aie.TileOp(I32Attr(1), I32Attr(3))
    tile23 = aie.TileOp(I32Attr(2), I32Attr(3))

    buf23 = aie.BufferOp(tile23, i32, ArrayAttr([I32Attr(256)]), StringAttr("a23"))

    kernel1 = func.FuncOp.external("extern_kernel1", [], [])
    kernel2 = func.FuncOp.external("extern_kernel2", [memref.MemRefType.from_element_type_and_shape(i32, [256])], [])

    lock13_2 = aie.LockOp(I32Attr(2), I32Attr(0), tile13, StringAttr("lock_a13_2"))

    @Builder.region
    def core13_region(builder: Builder):
        builder.insert(func.Call(kernel1.sym_name.data, [], []))
        builder.insert(aie.EndOp())

    core13 = aie.CoreOp(I32Attr(1), tile13, core13_region, "kernel1.o")

    @Builder.region
    def core23_region(builder: Builder):
        uselock13_2_acquire = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock13_2)
        builder.insert(uselock13_2_acquire)
        builder.insert(func.Call(kernel2.sym_name.data, [buf23], []))
        uselock13_2_release = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock13_2)
        builder.insert(uselock13_2_release)
        builder.insert(aie.EndOp())

    core23 = aie.CoreOp(I32Attr(1), tile23, core23_region, "kernel2.o")

    module = ModuleOp([tile13, tile23, buf23, kernel1, kernel2, lock13_2, core13, core23])
    print(module)

In [3]:
tutorial8()

builtin.module {
  %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 3 : i32} : () -> index
  %1 = "AIE.tile"() {"col" = 2 : i32, "row" = 3 : i32} : () -> index
  %2 = "AIE.buffer"(%1) {"sym_name" = "a23"} : (index) -> memref<256xi32>
  func.func private @extern_kernel1() -> ()
  func.func private @extern_kernel2(memref<256xi32>) -> ()
  %3 = "AIE.lock"(%0) {"lockID" = 2 : i32, "init" = 0 : i32, "sym_name" = "lock_a13_2"} : (index) -> index
  %4 = AIE.core (%0) {
    func.call @extern_kernel1() : () -> ()
    "AIE.end"() : () -> ()
  } { link_with="kernel1.o" }
  %5 = AIE.core (%1) {
    AIE.useLock(%3, "Acquire", 0)
    func.call @extern_kernel2(%2) : (memref<256xi32>) -> ()
    AIE.useLock(%3, "Release", 1)
    "AIE.end"() : () -> ()
  } { link_with="kernel2.o" }
}
